In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn import preprocessing
import keras
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction import FeatureHasher
from scipy.stats import skew
from scipy.stats import boxcox
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from collections import Counter, defaultdict
from keras.preprocessing.sequence import pad_sequences
from tk_utils.get_time_cat import get_time_cat

Using TensorFlow backend.


In [2]:
import gc

In [3]:
# jtplot.style(theme='onedork')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time']

In [5]:
data_path = "/Users/chaoxu/code/ml_more_algorithm/Data/talking_data"
train_data = "{0}/{1}".format(data_path, "train_sample.csv")
test_data = "{0}/{1}".format(data_path, "test.csv")

In [6]:
df = pd.read_csv(train_data,parse_dates=["click_time",'attributed_time'])

In [7]:
df.head(5)

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaT,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaT,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaT,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaT,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaT,0


In [8]:
df_train = pd.read_csv(train_data,parse_dates=["click_time"],usecols=train_cols)

In [9]:
df_test = pd.read_csv(test_data,parse_dates=["click_time"],usecols=test_cols)

In [10]:
df_train.shape

(100000, 7)

In [11]:
df_train.dtypes

ip                        int64
app                       int64
device                    int64
os                        int64
channel                   int64
click_time       datetime64[ns]
is_attributed             int64
dtype: object

In [12]:
df_train.is_attributed.value_counts(normalize=True)

0    0.99773
1    0.00227
Name: is_attributed, dtype: float64

In [15]:
df_train.values.max(0)

array([364757, 551, 3867, 866, 498, Timestamp('2017-11-09 15:59:51'), 1],
      dtype=object)

In [13]:
df_test.dtypes

ip                     int64
app                    int64
device                 int64
os                     int64
channel                int64
click_time    datetime64[ns]
dtype: object

In [15]:
df_test['day'] = df_test['click_time'].dt.day.astype('uint8')

In [16]:
df_test.day.value_counts()

10    18790469
Name: day, dtype: int64

In [11]:
# df_train.set_index('click_time').sort_index()

In [12]:
df_train = df_train.append(df_test)
del df_test
gc.collect()

0

In [13]:
df_train.dtypes

app                       int64
channel                   int64
click_time       datetime64[ns]
device                    int64
ip                        int64
is_attributed           float64
os                        int64
dtype: object

In [14]:
df_train.head(5)

,app,channel,click_time,device,ip,is_attributed,os
0,12,497,2017-11-07 09:30:38,1,87540,0.0,13
1,25,259,2017-11-07 13:40:27,1,105560,0.0,17
2,12,212,2017-11-07 18:05:24,1,101424,0.0,19
3,13,477,2017-11-07 04:58:08,1,94584,0.0,13
4,12,178,2017-11-09 09:00:09,1,68413,0.0,1


In [15]:
df1 = df_train.set_index('click_time').sort_index()

In [16]:
df1.head(10)

,app,channel,device,ip,is_attributed,os
click_time,,,,,,
2017-11-06 16:00:00,12,178,1,48646,0.0,19
2017-11-06 16:00:09,8,145,1,5314,0.0,13
2017-11-06 16:00:09,12,328,1,93836,0.0,30
2017-11-06 16:00:11,23,153,1,73954,0.0,19
2017-11-06 16:00:11,3,135,1,91574,0.0,17
2017-11-06 16:00:13,2,205,1,25705,0.0,19
2017-11-06 16:00:19,15,130,1,93057,0.0,19
2017-11-06 16:00:20,15,153,1,87868,0.0,19
2017-11-06 16:00:20,12,178,1,106524,0.0,28


In [17]:
df1.index = pd.to_datetime(df1.index)

In [ ]:
df2 = df1.groupby(['ip','app'])['os'].rolling('1d').apply(lambda x: np.unique(x).shape[0])

In [1]:
# df2.reset_index(['ip','app'],drop=True).sort_index()

In [25]:
# df_train.columns.tolist()

['app', 'channel', 'click_time', 'device', 'ip', 'is_attributed', 'os']